In [3]:
import os
import re
from tqdm import tqdm
import transformers, torch
from transformers import AutoModelForCausalLM, AutoTokenizer, LlamaTokenizer, LlamaForCausalLM, GenerationConfig

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# tokenizer = AutoTokenizer.from_pretrained("Salesforce/codet5p-2b")
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-1")

model = AutoModelForCausalLM.from_pretrained("microsoft/phi-1",
                                              trust_remote_code=True,
                                              torch_dtype=torch.float16,
                                              low_cpu_mem_usage=True)

model.to(DEVICE)


C:\Users\tyfann\anaconda3\envs\phi\lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(51200, 2048)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-23): 24 x PhiDecoderLayer(
        (self_attn): PhiAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (k_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (v_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (dense): Linear(in_features=2048, out_features=2048, bias=True)
          (rotary_emb): PhiRotaryEmbedding()
        )
        (mlp): PhiMLP(
          (activation_fn): NewGELUActivation()
          (fc1): Linear(in_features=2048, out_features=8192, bias=True)
          (fc2): Linear(in_features=8192, out_features=2048, bias=True)
        )
        (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.0, inplace=False)
      )
    )
    (final_layernorm): LayerNorm((2048,), e

In [4]:
max_len = 600

In [7]:
def GEN_SOLUTION(prompt):
    generation_config = transformers.GenerationConfig(
        do_sample=True,
        temperature=0.2,
        top_p=0.95,
        max_new_tokens = max_len,
    )

    model_inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=max_len).to(DEVICE)

    generated_ids = model.generate(**model_inputs)
    return tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [8]:
from evalplus.data import get_human_eval_plus, write_jsonl

samples = [
    dict(task_id=task_id, solution=GEN_SOLUTION(problem["prompt"]))
    for task_id, problem in get_human_eval_plus().items()
]
write_jsonl("phi_1_samples.jsonl", samples)

C:\Users\tyfann\anaconda3\envs\phi\lib\site-packages\transformers\generation\utils.py:1133: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
C:\Users\tyfann\anaconda3\envs\phi\lib\site-packages\transformers\generation\utils.py:1141: UserWarning: Input length of input_ids is 127, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(
C:\Users\tyfann\anaconda3\envs\phi\lib\site-packages\transformers\generation\utils.py:1141: UserWarning: Input length of input_ids is 90, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(
C:\Users\tyfann\anaconda3\envs\phi\lib\site-packages\transformers\generation\utils.py:1141: UserWarning: Input length of input_ids is 121, but `max_length` is